# 2. OpenAI の チャット API の基礎


## 2.3. 入出力の長さの制限や料金に影響する「トークン」


### トークン


In [ ]:
!pip install tiktoken==0.7.0

In [ ]:
import tiktoken

text = "ChatGPT"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
for token in tokens:
    print(encoding.decode([token]))

### Tokenizer と tiktoken の紹介


In [ ]:
import tiktoken

text = "LLMを使ってクールなものを作るのは簡単だが、プロダクションで使えるものを作るのは非常に難しい。"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
print(len(tokens))

## 2.4. Chat Completions API を試す環境の準備


### OpenAI の API キーの準備


In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## 2.5. Chat Completions API のハンズオン


### OpenAI のライブラリ


#### 【注意】既知のエラーについて

openai パッケージが依存する httpx のアップデートにより、`openai==1.40.6` を使用する箇所で `TypeError: Client.__init__() got an unexpected keyword argument 'proxies'` というエラーが発生するようになりました。

このエラーは、`!pip install httpx==0.27.2` のように、httpx の特定バージョンをインストールすることで回避することができます。

なお、Google Colab で一度上記のエラーに遭遇したあとで `!pip install httpx==0.27.2` のようにパッケージをインストールし直した場合、以下のどちらかの操作を実施する必要があります。

- Google Colab の「ランタイム」から「セッションを再起動する」を実行する
- 「ランタイムを接続解除して削除」を実行してパッケージのインストールからやり直す


In [1]:
!pip install openai==1.40.6 httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: openai
    Found existing installation: openai 1.91.0
    Uninstalling openai-1.91.0:
      Successfully uninstalled openai-1.91.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which is incompatible.
google-genai 1.21.1 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


### Chat Completions API の呼び出し


In [3]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"},
    ],
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-BngZLWFIBsd05nkYdDa6Qu3ykzffE",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことについてお話ししましょうか？",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  "created": 1751181407,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_62a23a81ef",
  "usage": {
    "completion_tokens": 27,
    "prompt_tokens": 25,
    "total_tokens": 52,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}


### 会話履歴を踏まえた応答を得る


In [4]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"},
        {"role": "assistant", "content": "こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことをお話ししましょうか？"},
        {"role": "user", "content": "私の名前が分かりますか？"},
    ],
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-BngZPCLC76AWejUd52n4obEtDiJPB",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "はい、ジョンさんですね！あなたのお名前を教えていただきました。ほかに何かお話ししたいことがありますか？",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  "created": 1751181411,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_34a54ae93c",
  "usage": {
    "completion_tokens": 31,
    "prompt_tokens": 69,
    "total_tokens": 100,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}


### ストリーミングで応答を得る


In [5]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"},
    ],
    stream=True,
)

for chunk in response:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end="", flush=True)

こんにちは、ジョンさん！お会いできてうれしいです。今日はどんなことをお話ししましょうか？

### JSON モード


In [6]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": '人物一覧を次のJSON形式で出力してください。\n{"people": ["aaa", "bbb"]}',
        },
        {
            "role": "user",
            "content": "昔々あるところにおじいさんとおばあさんがいました",
        },
    ],
    response_format={"type": "json_object"},
)
print(response.choices[0].message.content)

{"people": ["おじいさん", "おばあさん"]}


### Vision（画像入力）


In [7]:
from openai import OpenAI  # OpenAIライブラリを使うよ

client = OpenAI()  # AIと話すための電話を作るよ

image_url = "https://raw.githubusercontent.com/yoshidashingo/langchain-book/main/assets/cover.jpg"  # 説明してもらいたい画像のURLだよ

response = client.chat.completions.create(  # AIに画像を送って説明してもらうよ
    model="gpt-4o-mini",  # gpt-4o は画像も見られるモデルだよ（miniは画像非対応💦）
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "画像を説明してください。"},  # ユーザーからのお願い「この画像なに？」
                {"type": "image_url", "image_url": {"url": image_url}},  # 実際に見てほしい画像のURLを渡すよ
            ],
        }
    ],
)

print(response.choices[0].message.content)  # AIの返事を画面に表示するよ


この画像は書籍の表紙です。タイトルは「ChatGPT/LangChainによるチャットシステム構築【実践】入門」となっています。著者は吉田真吾さんと大場秀樹さんです。表紙には色とりどりのオウムのイラストが描かれており、デザインには青色が基調として使われています。書籍はChatGPTやLangChainに関連した内容をテーマにしているようです。


### （コラム）Completions API


In [8]:
from openai import OpenAI

client = OpenAI()

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt="こんにちは！私はジョンと言います！",
)
print(response.to_json(indent=2))

{
  "id": "cmpl-BngbUOnCXhBRIctmtcGOSHgMoci8N",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\nこんにちは、ジョンさん！どんなことに興"
    }
  ],
  "created": 1751181540,
  "model": "gpt-3.5-turbo-instruct:20230824-v2",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 11,
    "total_tokens": 27
  }
}


## 2.6. Function calling


### Function calling のサンプルコード


In [11]:
import json  # JSONというデータの形を使うための道具を用意するよ（辞書みたいなデータの形）

def get_current_weather(location, unit="fahrenheit"):  # 場所（都市）と温度の単位を受け取る関数だよ。デフォルトは「華氏」
    if "tokyo" in location.lower():  # 入力された場所が「tokyo」なら（大文字小文字は無視するよ）
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})  # 東京の気温10を返す（unitは選べるよ）
    elif "san francisco" in location.lower():  # サンフランシスコだったら
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})  # サンフランシスコの気温を返す
    elif "paris" in location.lower():  # パリだったら
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})  # パリの気温を返す
    else:  # 上のどれでもなかったら（知らない場所だったら）
        return json.dumps({"location": location, "temperature": "unknown"})  # 気温は「わかりません」と返すよ


In [12]:
tools = [  # ツール（使える関数）をまとめたリストを作るよ
    {
        "type": "function",  # これは「関数（function）」の種類だよって教えてる
        "function": {
            "name": "get_current_weather",  # 関数の名前だよ（AIに「この名前の関数があるよ」と伝える）
            "description": "Get the current weather in a given location",  # 関数の説明（どんなことをする関数？）
            "parameters": {  # 関数に渡すパラメータ（＝お願いに必要な情報）を教えるよ
                "type": "object",  # 入力はオブジェクト（辞書のような形）だよ
                "properties": {
                    "location": {  # 「どこ？」って場所のこと
                        "type": "string",  # 場所は文字（例："Tokyo"）で書いてね
                        "description": "The city and state, e.g. San Francisco, CA",  # 「どんな場所？」の説明だよ
                    },
                    "unit": {  # 温度の単位だよ
                        "type": "string",  # 単位も文字で書くよ
                        "enum": ["celsius", "fahrenheit"]  # 選べるのは「摂氏」か「華氏」だけだよ
                    },
                },
                "required": ["location"],  # 「場所」は必ず入れてね！（unitはなくてもOK）
            },
        },
    }
]


In [13]:
from openai import OpenAI  # OpenAIという会社の道具を使えるようにするよ（AIと話すための準備）

client = OpenAI()  # OpenAIとつながるための電話を作っているよ（AIと会話する入口）

messages = [  # AIに送るメッセージをリストで用意するよ
    {"role": "user", "content": "東京の天気はどうですか？"},  # ユーザーが「東京の天気は？」と質問しているよ
]

response = client.chat.completions.create(  # ChatGPTに質問して返事をもらうための命令だよ
    model="gpt-4o",  # GPT-4oというかしこいAIを使うよ（画像や音声もわかる強い子！）
    messages=messages,  # さっき用意した質問をここで使うよ
    tools=tools,  # 天気を調べるための道具（tools）をAIに渡しているよ
)

print(response.to_json(indent=2))  # AIからの返事をきれいに見やすい形で画面に表示するよ


{
  "id": "chatcmpl-BngfnkIB06jpVFCNdnUu3EpFKi2qr",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_eRgobq09rR65AMhsoQ49rFDf",
            "function": {
              "arguments": "{\"location\":\"Tokyo, Japan\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ],
        "annotations": []
      }
    }
  ],
  "created": 1751181807,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_07871e2ad8",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 81,
    "total_tokens": 98,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tok

In [14]:
response_message = response.choices[0].message  # ChatGPTの返事の中で、一番最初のメッセージを取り出してるよ
messages.append(response_message.to_dict())  # その返事を辞書の形に変えて、次の会話に使うためリストに追加してるよ


In [15]:
available_functions = {  # どの関数が使えるかを辞書でまとめておくよ（名前と中身をセット）
    "get_current_weather": get_current_weather,  # たとえば「get_current_weather」って名前でこの関数が呼び出せるよ
}

# 使いたい関数が1つとは限らないから、全部チェックするよ
for tool_call in response_message.tool_calls:  # ChatGPTが「これ使って！」と言ってきたツール（関数）を1つずつ見るよ
    function_name = tool_call.function.name  # 使う関数の名前を取り出すよ（たとえば "get_current_weather"）
    function_to_call = available_functions[function_name]  # その名前に対応する関数を取り出すよ
    function_args = json.loads(tool_call.function.arguments)  # ChatGPTから送られた「引数」を辞書の形に読み取るよ

    function_response = function_to_call(  # 実際に関数を実行するよ
        location=function_args.get("location"),  # 「場所」の情報を渡して
        unit=function_args.get("unit"),  # 「摂氏 or 華氏」の情報も渡すよ
    )
    print(function_response)  # 関数の結果（たとえば天気）を画面に表示するよ

    messages.append(  # 関数の実行結果を会話履歴に追加して、ChatGPTが続きを話せるようにするよ
        {
            "tool_call_id": tool_call.id,  # どのツールの返事なのかをつなげて記録するよ
            "role": "tool",  # これはツールからの返事だよ、っていう役割の名前
            "name": function_name,  # どの関数からの返事か
            "content": function_response,  # 実際の返事の中身（たとえば天気情報）
        }
    )


{"location": "Tokyo", "temperature": "10", "unit": null}


In [16]:
print(json.dumps(messages, ensure_ascii=False, indent=2))  # 今までの会話のやりとり（messages）を見やすい形にして画面に出すよ（日本語もそのまま見えるようにするよ）


[
  {
    "role": "user",
    "content": "東京の天気はどうですか？"
  },
  {
    "content": null,
    "refusal": null,
    "role": "assistant",
    "tool_calls": [
      {
        "id": "call_eRgobq09rR65AMhsoQ49rFDf",
        "function": {
          "arguments": "{\"location\":\"Tokyo, Japan\"}",
          "name": "get_current_weather"
        },
        "type": "function"
      }
    ],
    "annotations": []
  },
  {
    "tool_call_id": "call_eRgobq09rR65AMhsoQ49rFDf",
    "role": "tool",
    "name": "get_current_weather",
    "content": "{\"location\": \"Tokyo\", \"temperature\": \"10\", \"unit\": null}"
  }
]


In [17]:
second_response = client.chat.completions.create(  # さっきまでの会話の続きとして、もう一度ChatGPTに返事をお願いするよ
    model="gpt-4o",  # GPT-4oっていうかしこいAIを使うよ
    messages=messages,  # 今までの会話の全部をまとめて渡して、「次どう答える？」と聞いてるよ
)
print(second_response.to_json(indent=2))  # ChatGPTの返事を、読みやすい形で画面に表示するよ（中身をじっくり見るため）


{
  "id": "chatcmpl-BngkV9fQdyQm398nJYxub5imy97Ce",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "東京の現在の気温は約10度です。詳細な天気情報を確認するには、天気予報のアプリやウェブサイトを参照してください。",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  "created": 1751182099,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_07871e2ad8",
  "usage": {
    "completion_tokens": 38,
    "prompt_tokens": 59,
    "total_tokens": 97,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}
